In [15]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

from utils.data import extract_eeg_data, multi_to_binary_classification, split_dataset, dict_to_arr
from utils.labels import get_pss_labels, get_stai_labels
from utils.valid_recs import get_valid_recs
from features import all_features, hjorth_features
from classifiers import knn_classification, svm_classification, cnn_classification, EEGNet_classification
import utils.variables as v

from tensorflow.keras import utils as np_utils


In [2]:
valid_recs = get_valid_recs(data_type='ica', output_type = 'np')
print(f'Valid recs {valid_recs}')

x_dict_ = extract_eeg_data(valid_recs, data_type='ica', output_type='np')


ERROR:root:1) Failed to read data for recording P006_S002_001
ERROR:root:1) Failed to read data for recording P006_S002_002


Filtering out invalid recordings

Data/ICA_data\sub-P010_ses-S001_run-001.mat not valid
Data/ICA_data\sub-P013_ses-S001_run-001.mat not valid
Data/ICA_data\sub-P013_ses-S001_run-002.mat not valid


ERROR:root:1) Failed to read data for recording P028_S001_001
ERROR:root:1) Failed to read data for recording P028_S001_002


Data/ICA_data\sub-P020_ses-S001_run-001.mat not valid
Data/ICA_data\sub-P023_ses-S002_run-002.mat not valid
Returning valid recordings

Valid recs ['P001_S001_001', 'P001_S001_002', 'P001_S002_001', 'P001_S002_002', 'P002_S001_001', 'P002_S001_002', 'P002_S002_001', 'P002_S002_002', 'P003_S001_001', 'P003_S001_002', 'P003_S002_001', 'P003_S002_002', 'P004_S001_001', 'P004_S001_002', 'P004_S002_001', 'P004_S002_002', 'P005_S001_001', 'P005_S001_002', 'P005_S002_001', 'P005_S002_002', 'P006_S001_001', 'P006_S001_002', 'P007_S001_001', 'P007_S001_002', 'P007_S002_001', 'P007_S002_002', 'P008_S001_001', 'P008_S001_002', 'P008_S002_001', 'P008_S002_002', 'P009_S001_001', 'P009_S001_002', 'P009_S002_001', 'P009_S002_002', 'P010_S001_002', 'P010_S002_001', 'P010_S002_002', 'P011_S001_001', 'P011_S001_002', 'P011_S002_001', 'P011_S002_002', 'P012_S001_001', 'P012_S001_002', 'P012_S002_001', 'P012_S002_002', 'P013_S002_001', 'P013_S002_002', 'P014_S001_001', 'P014_S001_002', 'P014_S002_001', 'P

In [3]:
y_dict_ = get_stai_labels(valid_recs) 
#y_dict = get_pss_labels(valid_recs)

    SubjectNo  D1Y1  D2Y1  J1Y1  J2Y1
0           1    26    30    29    31
1           2    38    41    26    34
2           3    58    56    36    35
3           4    40    45    24    24
4           5    25    31    38    37
5           6    49    58     0     0
6           7    56    50    28    28
7           8    46    37    23    27
8           9    41    47    27    22
9          10    37    20    23    21
10         11    50    49    31    47
11         12    42    47    47    41
12         13    35    35    28    33
13         14    54    35    26    26
14         15    51    55    33    42
15         16    35    38    42    45
16         17    37    35    24    20
17         18    54    62    41    48
18         19    47    52    30    36
19         20    46    38    24    25
20         21    44    54    33    39
21         22    49    51    28    34
22         23    56    53    33    28
23         24    52    58    36    41
24         25    48    62    29    56
25         2

In [4]:
print(len(x_dict_))
print(len(y_dict_))


103
103


In [5]:
x_dict, y_dict = multi_to_binary_classification(x_dict_, y_dict_)


The extracted keys : 
['P002_S001_001', 'P002_S001_002', 'P004_S001_001', 'P004_S001_002', 'P005_S002_001', 'P005_S002_002', 'P008_S001_002', 'P009_S001_001', 'P012_S001_001', 'P012_S002_002', 'P015_S002_002', 'P016_S001_002', 'P016_S002_001', 'P016_S002_002', 'P017_S001_001', 'P018_S002_001', 'P020_S001_002', 'P021_S001_001', 'P021_S002_002', 'P024_S002_002', 'P026_S001_001', 'P026_S001_002', 'P027_S001_002', 'P027_S002_001']

Dictionary after removal of keys from y_dict: 
 dict_keys(['P001_S001_001', 'P001_S001_002', 'P001_S002_001', 'P001_S002_002', 'P002_S002_001', 'P002_S002_002', 'P003_S001_001', 'P003_S001_002', 'P003_S002_001', 'P003_S002_002', 'P004_S002_001', 'P004_S002_002', 'P005_S001_001', 'P005_S001_002', 'P006_S001_001', 'P006_S001_002', 'P007_S001_001', 'P007_S001_002', 'P007_S002_001', 'P007_S002_002', 'P008_S001_001', 'P008_S002_001', 'P008_S002_002', 'P009_S001_002', 'P009_S002_001', 'P009_S002_002', 'P010_S001_002', 'P010_S002_001', 'P010_S002_002', 'P011_S001_001'

In [6]:
train_data_dict, test_data_dict, val_data_dict, train_labels_dict, test_labels_dict, val_labels_dict = split_dataset(x_dict, y_dict)

In [19]:
train_data_arr = dict_to_arr(train_data_dict)
test_data_arr = dict_to_arr(test_data_dict)
val_data_arr = dict_to_arr(val_data_dict)

train_labels_arr = np.reshape(np.array(list(train_labels_dict.values())), (len(train_data_arr),1))
test_labels_arr = np.reshape(np.array(list(test_labels_dict.values())), (len(test_data_arr),1))
val_labels_arr = np.reshape(np.array(list(val_labels_dict.values())), (len(val_data_arr),1))

train_labels_arr[train_labels_arr == 2] = 1
test_labels_arr[test_labels_arr == 2] = 1
val_labels_arr[val_labels_arr == 2] = 1

# convert labels to one-hot encodings.
#train_labels_arr = np_utils.to_categorical(train_labels_arr-1)
#val_labels_arr   = np_utils.to_categorical(val_labels_arr-1)
#test_labels_arr  = np_utils.to_categorical(test_labels_arr-1)

# Our vectorized labels
train_labels_arr = np.asarray(train_labels_arr).astype('float32').reshape((-1,1))
test_labels_arr  = np.asarray(test_labels_arr).astype('float32').reshape((-1,1))
val_labels_arr  = np.asarray(val_labels_arr).astype('float32').reshape((-1,1))


In [20]:
EEGNet_classification(train_data_arr, test_data_arr, val_data_arr, train_labels_arr, test_labels_arr, val_labels_arr)

Epoch 1/300


ValueError: in user code:

    File "c:\Users\annej\anaconda3\envs\MNE\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\annej\anaconda3\envs\MNE\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\annej\anaconda3\envs\MNE\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\annej\anaconda3\envs\MNE\lib\site-packages\keras\engine\training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\annej\anaconda3\envs\MNE\lib\site-packages\keras\engine\training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "c:\Users\annej\anaconda3\envs\MNE\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_v